# AgentCore Runtime での大規模マルチモーダルペイロードの処理

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime が Excel ファイルや画像などのマルチモーダルコンテンツを含む最大 100MB の大規模ペイロードを処理する方法を示します。AgentCore Runtime は、リッチメディアコンテンツと大規模データセットをシームレスに処理するように設計されています。

### チュートリアル詳細

|情報| 詳細|
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 大規模ペイロード & マルチモーダル処理|
| エージェントタイプ   | シングル         |
| エージェントフレームワーク | Strands Agents |
| LLM モデル          | Anthropic Claude Haiku 4.5 |
| チュートリアルコンポーネント | 大規模ファイル処理、画像分析、Excel データ処理 |
| チュートリアル分野   | データ分析 & マルチモーダル AI                                                   |
| 難易度              | 中級                                                                     |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK|

### 主な機能

* **大規模ペイロードサポート**: 最大 100MB のファイルを処理
* **マルチモーダル処理**: Excel ファイル、画像、テキストを同時に処理
* **データ分析**: 構造化データと視覚コンテンツからインサイトを抽出
* **Base64 エンコーディング**: JSON ペイロードを通じてバイナリデータを安全に送信

## 前提条件

* Python 3.10+
* AWS 認証情報が設定済み
* Docker が実行中
* テスト用のサンプル Excel ファイルと画像

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## サンプルデータファイルの作成

大規模ペイロードの処理を示すために、サンプルの Excel ファイルと画像ファイルを作成しましょう：

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os

# サンプル売上データを含む大規模 Excel ファイルを作成
np.random.seed(42)
data = {
    'Date': pd.date_range('2023-01-01', periods=1000, freq='h'),
    'Product': np.random.choice(['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y'], 1000),
    'Sales': np.random.randint(1, 1000, 1000),
    'Revenue': np.random.uniform(10.0, 5000.0, 1000),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], 1000),
    'Customer_ID': np.random.randint(1000, 9999, 1000)
}

df = pd.DataFrame(data)
df.to_excel('large_sales_data.xlsx', index=False)

# サンプルチャート画像を作成
img = Image.new('RGB', (600, 500), color='white')
draw = ImageDraw.Draw(img)

# シンプルな棒グラフを描画
products = ['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y']
values = [250, 180, 320, 150, 280]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']

max_value = max(values)
bar_width = 120
start_x = 100

for i, (product, value, color) in enumerate(zip(products, values, colors)):
    x = start_x + i * (bar_width + 20)
    height = int((value / max_value) * 400)
    y = 500 - height
    
    # 棒を描画
    draw.rectangle([x, y, x + bar_width, 500], fill=color)
    
    # ラベルを追加（フォントなしで簡略化）
    draw.text((x + 10, 510), product[:8], fill='black')
    draw.text((x + 10, y - 20), str(value), fill='black')

draw.text((300, 50), 'Sales Performance by Product', fill='black')
img.save('sales_chart.png')

# ファイルサイズを確認
excel_size = os.path.getsize('large_sales_data.xlsx') / (1024 * 1024)  # MB
image_size = os.path.getsize('sales_chart.png') / (1024 * 1024)  # MB

print(f"Excel ファイルサイズ: {excel_size:.2f} MB")
print(f"画像ファイルサイズ: {image_size:.2f} MB")
print(f"合計ペイロードサイズ: {excel_size + image_size:.2f} MB")

## マルチモーダルエージェントの作成

Excel ファイルと大規模ペイロードからの画像の両方を処理できるエージェントを作成しましょう：

In [ ]:
%%writefile multimodal_data_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
import pandas as pd
import base64
import io
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# モデルとエージェントを初期化
model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
    max_tokens=16000
)

agent = Agent(
    model=model,
    system_prompt="""
    あなたは大規模な Excel ファイルと画像を処理できるデータ分析アシスタントです。
    マルチモーダルデータが与えられた場合、構造化データと視覚コンテンツの両方を分析し、
    両方のデータソースを組み合わせた包括的なインサイトを提供してください。
    """
)

@app.entrypoint
def multimodal_data_processor(payload, context):
    """
    Excel データと画像を含む大規模マルチモーダルペイロードを処理します。
    
    Args:
        payload: prompt、excel_data（base64）、image_data（base64）を含むデータ
        context: Runtime コンテキスト情報
    
    Returns:
        str: 両方のデータソースからの分析結果
    """
    prompt = payload.get("prompt", "Analyze the provided data.")
    excel_data = payload.get("excel_data", "")
    image_data = payload.get("image_data", "")
    
    print(f"=== 大規模ペイロード処理 ===")
    print(f"セッション ID: {context.session_id}")
    
    if excel_data:
        print(f"Excel データサイズ: {len(excel_data) / 1024 / 1024:.2f} MB")
    if image_data:
        print(f"画像データサイズ: {len(image_data) / 1024 / 1024:.2f} MB")
    print(f"Excel データ: {excel_data}")
    print(f"画像データ: {image_data}")
    print(f"=== 処理開始 ===")
    # base64 をバイトにデコード
    excel_bytes = base64.b64decode(excel_data)
    # base64 をバイトにデコード
    image_bytes = base64.b64decode(image_data)
    
    # データコンテキストを追加した拡張プロンプト
    enhanced_prompt = f"""{prompt}
    Please analyze both data sources and provide insights.
    """
    
    response = agent(
        [{
            "document": {
                "format": "xlsx",
                "name": "excel_data",
                "source": {
                    "bytes": excel_bytes
                }
            }
        },
        {
            "image": {
                "format": "png",
                "source": {
                    "bytes": image_bytes
                }
            }
        },
        {
            "text": enhanced_prompt
        }]
    )
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## インフラストラクチャのセットアップとエージェントのデプロイ

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="multimodal_data_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="multimodal_data_agent"
)

launch_result = agentcore_runtime.launch()

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"デプロイメントステータス: {status}")

print(f"最終ステータス: {status}")

## 大規模マルチモーダルペイロードのテスト

Excel データと画像の両方を含む大規模ペイロードでエージェントをテストしましょう：

In [ ]:
import base64
import uuid
import json
from IPython.display import Markdown, display

# ファイルを base64 にエンコード
with open('large_sales_data.xlsx', 'rb') as f:
    excel_base64 = base64.b64encode(f.read()).decode('utf-8')

with open('sales_chart.png', 'rb') as f:
    image_base64 = base64.b64encode(f.read()).decode('utf-8')

# 大規模ペイロードを作成
large_payload = {
    "prompt": "Analyze the sales data from the Excel file and correlate it with the chart image. Provide insights on sales performance and trends.",
    "excel_data": excel_base64,
    "image_data": image_base64
}

session_id = str(uuid.uuid4())
print(f"📊 大規模マルチモーダルペイロードを処理中...")
print(f"📋 セッション ID: {session_id}")
print(f"📄 Excel サイズ: {len(excel_base64) / 1024 / 1024:.2f} MB")
print(f"🖼️ 画像サイズ: {len(image_base64) / 1024 / 1024:.2f} MB")
print(f"📦 合計ペイロード: {len(json.dumps(large_payload)) / 1024 / 1024:.2f} MB\n")

# 大規模ペイロードでエージェントを呼び出し
invoke_response = agentcore_runtime.invoke(
    large_payload,
    session_id=session_id
)
final_response = ""
for r in invoke_response['response']:
    final_response += r
response_data = final_response
display(Markdown(response_data))

## リソースのクリーンアップ

In [ ]:
import boto3

# AWS リソースをクリーンアップ
agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)

# AgentCore Runtime を削除
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# ECR リポジトリを削除
ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# ローカルファイルをクリーンアップ
os.remove('large_sales_data.xlsx')
os.remove('sales_chart.png')

print("✅ クリーンアップ完了!")

# おめでとうございます！

Amazon Bedrock AgentCore Runtime での大規模マルチモーダルペイロードの処理を正常にデモンストレーションしました！

## 学んだこと：

### 大規模ペイロード処理
* **100MB サポート**: AgentCore Runtime は最大 100MB のペイロードを処理可能
* **Base64 エンコーディング**: JSON ペイロードを通じてバイナリデータを安全に送信
* **効率的な処理**: 大規模データ処理に最適化された Runtime

### マルチモーダル機能
* **Excel 分析**: スプレッドシートからの構造化データを処理
* **画像処理**: 視覚コンテンツとチャートを分析
* **統合分析**: 複数のデータタイプからインサイトを相関付け

### 主なメリット
* **リッチデータ処理**: 複雑なマルチフォーマットデータセットを処理
* **スケーラブルアーキテクチャ**: 大規模ワークロード向けに設計された Runtime
* **ツール統合**: 専門的なデータ処理のためのカスタムツール
* **エンタープライズ対応**: 機密性の高いビジネスデータの安全な処理

これは、複数のデータモダリティを持つエンタープライズスケールのデータ処理タスクを処理する AgentCore Runtime の能力を示しており、複雑なビジネスインテリジェンスとデータ分析アプリケーションに最適です。